In [ ]:
!pip install graphviz
!pip install anytree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from anytree import Node as Node_anytree, RenderTree
from anytree.exporter import DotExporter


In [ ]:
data = pd.read_csv("Restuarant.csv")
data.head(12)

,alternative,bar,friday,hungry,patron,price,rain,reservation,type,est,Goal
0,Yes,no,no,yes,some,3,no,yes,French,0 -10,1
1,Yes,no,no,yes,full,1,no,no,Thai,30 - 60,0
2,no,yes,no,no,some,1,no,no,Burger,0 - 10,1
3,Yes,no,yes,yes,full,1,yes,no,Thai,10_30,1
4,Yes,no,yes,no,full,3,no,yes,French,60>,0
5,no,yes,no,yes,some,2,yes,yes,Italian,0 -10,1
6,no,yes,no,no,None,1,yes,no,Burger,0 -10,0
7,no,no,no,yes,some,2,yes,yes,Thai,0 -10,1
8,no,yes,yes,no,full,1,yes,no,Burger,60>,0
9,Yes,yes,yes,yes,full,3,no,yes,Italian,10_30,0


In [ ]:
column_names = list(data.columns.values)
print(column_names)

['alternative', 'bar', 'friday', 'hungry', 'patron', 'price', 'rain', 'reservation', 'type', 'est', 'Goal']


In [ ]:
#with this function I make tree that can I get image from it by anytree library
#At first I make my tree to dictionary and then I make tree that its node from anytree node .
def make_node(dictionary_tree,father=None):
   
    if dictionary_tree['child'] == None:
        return Node_anytree(dictionary_tree['text'],parent=father)
    else:
        
        if father is None:
               root =  Node_anytree(dictionary_tree['text'])
        else:
               root = Node_anytree(dictionary_tree['text'],parent=father)
        
        for item in dictionary_tree['child']:
            make_node(item,father=root)
    
    return root

In [ ]:
class Node:
    def __init__(self,feature=None, choice_from_father=None, list_of_childs = None, value=None):
        self.feature = feature #name of feature in this tree make must difernce in child and father Entropy
        self.choice_from_father = choice_from_father #Decision that cause to go this node
        self.childs = list_of_childs #list of catogorical child it have
        self.value = value #its value if it was leaf
    def is_leaf(self):
        return self.value is not None
    
    def show_node(self): # with this function I make dictionary from my tree
      enter = '\n'
      if self.is_leaf():
        return {"text":f"{self.choice_from_father} _ {self.value}","child":None}
      else :
        
        list_of_show_child =[]
        for child in self.childs:
            list_of_show_child.append(child.show_node())
        
        return {"text":f"{self.choice_from_father} _ {self.feature}","child": list_of_show_child}
    

    def make_plot(self,name="resturant_entropy.png"):

        show_node_in_dic = self.show_node()
        
        root = make_node(show_node_in_dic)

        DotExporter(root).to_picture(name)


class DecisionTree:
    def __init__(self,list_of_feature_name,max_depth=100):
        self.list_of_feature_name = list_of_feature_name #list of feature our tree have
        self.max_depth = max_depth #maximum deep we can go on tree
        self.first = True #Show us this tree have real node on its root or not
        self.root = Node
    def _is_finished(self, depth): # check the sitaution of contiuse makeing tree
        if (depth >= self.max_depth
            or self.n_class_labels == 1):
            return True
        return False
    
    def _entropy(self, y):#calculate entropy for use 

        proportions = np.bincount(y) / len(y) 
        entropy = -np.sum([p * np.log2(p) for p in proportions if p > 0])
        return entropy


    def _create_split(self, X, list_of_category):
    #it get list of catogorical attributes that one category have and then splite them by this attributes and save them in dictionary

        dic_of_variable = {}
        for item in list_of_category :
                idx = np.argwhere(X == item).flatten()
                dic_of_variable[item] = idx

        return dic_of_variable


    def _find_Remainder(self,dic_of_variable:dict,n,y):
      # find the Weighted size of the child entropy
      Remainder = 0
      for key in  dic_of_variable:

        Remainder += (len(dic_of_variable[key])/n) * self._entropy(y[dic_of_variable[key]])

      return Remainder

    def _information_gain(self, X, y, list_of_category):
        # find the difference between father entropy and its weighted size of the its child entropy
        Entropy = self._entropy(y)
        dic_of_variable = self._create_split(X, list_of_category)
        n = len(y)
        
        Remainder = self._find_Remainder(dic_of_variable,n,y)
        return Entropy - Remainder

    def _best_split(self, X, y, features):
        #this function find best feature can make greatest
        # difference between father entropy and its weighted
        # size of the its child entropy
        best_split_score = -np.inf
        best_feat=None
        best_list_of_category = None


        for feat in features:
            X_feat = X[:, feat]
            all_data_feat = self.data[:,feat]
            list_of_category = np.unique(all_data_feat)
            score = self._information_gain(X_feat, y, list_of_category)
            if score > best_split_score:
                    best_split_score = score
                    best_feat = feat
                    best_list_of_category = list_of_category

        return best_feat, best_list_of_category
    
    def tree_show(self):
      
        return self.root.show_node()

    def make_plot(self):

        return self.root.make_plot()

    
    def _find_most_common_label(self,numpy_array,father_most_commen_sample):

          # find most things raping in numpy array
          # if every item have equal frequency come back our issue. (father most common sample)
          unique_list, counts = np.unique(numpy_array, return_counts=True)

          if len(unique_list) == 1:
            return unique_list[0]

          dic_of_unique_count = dict(zip(unique_list, counts))
          
          dic_of_unique_count_sort = dict(sorted(dic_of_unique_count.items(), key=lambda item: item[1]))

          list_key = []
          list_count = []

          for key in dic_of_unique_count_sort:
              list_key.append(key)
              list_count.append(dic_of_unique_count_sort[key])

          if unique_list[-1] > unique_list[-2]:

              return list_key[-1]

          else :

              return father_most_commen_sample

    def _build_tree(self, X, y,map_feature_number:list,father_most_commen_sample,choice_from_father,depth=0):

        self.n_samples, self.n_features = X.shape
        self.n_class_labels = len(np.unique(y))
        

        if self.first == True:
            self.data = X
            self.first = False
            
        #find most label rapid in one node
        most_common_Label = self._find_most_common_label(y,father_most_commen_sample)

        # stopping criteria
        if self._is_finished(depth):
            if len(y) == 0 :
                return Node(choice_from_father = choice_from_father ,value=father_most_commen_sample)
            return Node(choice_from_father = choice_from_father , value=most_common_Label)

        # get best split
        best_feat, list_of_category = self._best_split(X, y, map_feature_number)
        map_feature_number.remove(best_feat)
        # grow children recursively
        dic_of_variable = self._create_split(X[:, best_feat], list_of_category)

        list_of_childs = []
        for key in dic_of_variable:
            
            if len(dic_of_variable[key]) != 0 :

                  list_of_childs.append(self._build_tree(X[dic_of_variable[key],:],
                                                    y[dic_of_variable[key]],
                                                    map_feature_number,
                                                    most_common_Label,
                                                    key,
                                                    depth + 1))

            else:

                  list_of_childs.append(Node(feature=None, choice_from_father=key, list_of_childs = None, value=most_common_Label))
              



        return Node(self.list_of_feature_name[best_feat], choice_from_father,list_of_childs)
    
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        
        for child in node.list_of_childs:
          if x[node.feature] == child.choice_from_father:
            return self._traverse_tree(x,child)
          else:
            raise 


    def fit(self, X, y,map_feature_number):
        #make tree with training data 
        map_feature_number = map_feature_number
        father_most_commen_sample = np.argmax(np.bincount(y))
        choice_from_father = "root"
        self.root = self._build_tree(X, y,map_feature_number,father_most_commen_sample,choice_from_father)

    def predict(self, X):
        #predict new item with model fit
        predictions = [self._traverse_tree(x, self.root) for x in X]
        return np.array(predictions)

In [ ]:
list_of_feature_name = list(data.columns.values)[:-1]
map_feature_number = [i for i in range(len(list_of_feature_name))]

x, y = data.iloc[:,0:10], data.iloc[:,-1]
train_x = x.to_numpy()
train_label =y.to_numpy()


In [ ]:
resturant_decision_tree_model = DecisionTree(list_of_feature_name,max_depth=10)
resturant_decision_tree_model.fit(train_x,train_label,map_feature_number)

x = resturant_decision_tree_model.tree_show()


In [ ]:
print(x)

{'text': 'root _ patron', 'child': [{'text': 'None _ 0', 'child': None}, {'text': 'full _ hungry', 'child': [{'text': 'no _ 0', 'child': None}, {'text': 'yes _ type', 'child': [{'text': 'Burger _ 1', 'child': None}, {'text': 'French _ 1', 'child': None}, {'text': 'Italian _ 0', 'child': None}, {'text': 'Thai _ friday', 'child': [{'text': 'no _ 0', 'child': None}, {'text': 'yes _ 1', 'child': None}]}]}]}, {'text': 'some _ 1', 'child': None}]}


In [ ]:
resturant_decision_tree_model.make_plot()